In [ ]:
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from time import sleep
from random import randint
from IPython.core.display import clear_output
from urllib.parse import quote
import chromedriver_autoinstaller

In [2]:
import selenium
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
opt = webdriver.ChromeOptions()
opt.add_argument("--start-maximized")

chromedriver_autoinstaller.install()
driver = webdriver.Chrome(options=opt)

In [33]:
def get_search_html(url):
    response = get(url)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    return html_soup

In [88]:
def get_movie_url(movie, year):
    url = r"https://www.rottentomatoes.com/search?search=" + quote(movie)
    movie_url = None
    print((movie, year))
    
    try:
        soup = get_search_html(url)
        
        mov = soup.find('search-page-media-row', releaseyear=year)
        if mov is not None:
            movie_url = mov.find('a')['href']
        
    except NoSuchElementException:
        print("Movie not found!!")
    
    return movie_url
    

In [69]:
def get_rt_scores(movie, year):
    movie_url = get_movie_url(movie, year)
    print(movie_url)
    a = np.nan
    c = np.nan

    if movie_url is not None:
        try:
            response = get(movie_url)
            html_soup = BeautifulSoup(response.text, 'html.parser')

            x = html_soup.find('score-board')

            a = x['audiencescore']
            c = x['tomatometerscore']
        except TypeError:
            errors.append(movie)
            print("TypeError raised")

        print("get_rt_scores\nMovie:{},critic:{},audience:{}".format(
            movie, c, a))

    return [c, a]

In [ ]:
def make_db(d):
    df = pd.DataFrame.from_dict(d, orient='index', columns=['rt_critic', 'rt_audience'])
    df.reset_index(inplace=True)
    df.rename(columns={'index':'movie'}, inplace=True)
    df.to_csv('rt.csv', index=False)
    return df

In [7]:
imdb_df = pd.read_csv('imdb_meta.csv')

In [8]:
imdb_df

,movie,year,runtime,genre,imdb,meta,imdb_votes
0,The Shawshank Redemption,1994,142 min,Drama,9.3,80,2360495
1,The Dark Knight,2008,152 min,"Action, Crime, Drama",9.0,84,2319761
2,Inception,2010,148 min,"Action, Adventure, Sci-Fi",8.8,74,2083710
3,Fight Club,1999,139 min,Drama,8.8,66,1868351
4,Pulp Fiction,1994,154 min,"Crime, Drama",8.9,94,1839615
...,...,...,...,...,...,...,...
8887,Falling Overnight,2011,86 min,"Drama, Romance",6.7,40,1002
8888,Fremde Haut,2005,97 min,Drama,7.0,60,1002
8889,Bam gua nat,2008,144 min,Drama,7.1,73,1001
8890,Awful Nice,2013,92 min,Comedy,4.9,42,1000


In [11]:
movies = imdb_df['movie']

In [12]:
years = imdb_df['year']

In [90]:
tomato = {}

In [91]:
for i in range(6642,8893):
            clear_output(wait=True)
            print("i:{}, movie:{}".format(i, movies[i]))
            a = get_rt_scores(movies[i], years[i])
            tomato[movies[i]] = a
            tom_df = make_db(tomato)

KeyError: 8892